---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 3

In this assignment you will explore text message data and create models to predict if a message is spam or not. 

In [126]:
import pandas as pd
import numpy as np

spam_data = pd.read_csv('spam.csv')

spam_data['target'] = np.where(spam_data['target']=='spam',1,0)
spam_data.head(10)

,text,target
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0
5,FreeMsg Hey there darling it's been 3 week's n...,1
6,Even my brother is not like to speak with me. ...,0
7,As per your request 'Melle Melle (Oru Minnamin...,0
8,WINNER!! As a valued network customer you have...,1
9,Had your mobile 11 months or more? U R entitle...,1


In [127]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(spam_data['text'], 
                                                    spam_data['target'], 
                                                    random_state=0)

### Question 1
What percentage of the documents in `spam_data` are spam?

*This function should return a float, the percent value (i.e. $ratio * 100$).*

In [128]:
def answer_one():
    
    
    return (spam_data['target']==1).sum()*100/spam_data.shape[0]

### Question 2

Fit the training data `X_train` using a Count Vectorizer with default parameters.

What is the longest token in the vocabulary?

*This function should return a string.*

In [129]:
from sklearn.feature_extraction.text import CountVectorizer

def answer_two():
    count=CountVectorizer()
    count.fit(X_train)
    maxlength=max(map(len,count.get_feature_names()))
    return [i for i in count.get_feature_names() if len(i)==maxlength][0]
    


### Question 3

Fit and transform the training data `X_train` using a Count Vectorizer with default parameters.

Next, fit a fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1`. Find the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [130]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score

def answer_three():
    count=CountVectorizer()
    X_train_vectorized=count.fit_transform(X_train)
    model=MultinomialNB(alpha=0.1)
    model.fit(X_train_vectorized,y_train)
    predictions=model.predict(count.transform(X_test))
    return roc_auc_score(y_test,predictions)
    


### Question 4

Fit and transform the training data `X_train` using a Tfidf Vectorizer with default parameters.

What 20 features have the smallest tf-idf and what 20 have the largest tf-idf?

Put these features in a two series where each series is sorted by tf-idf value and then alphabetically by feature name. The index of the series should be the feature name, and the data should be the tf-idf.

The series of 20 features with smallest tf-idfs should be sorted smallest tfidf first, the list of 20 features with largest tf-idfs should be sorted largest first. 

*This function should return a tuple of two series
`(smallest tf-idfs series, largest tf-idfs series)`.*

In [131]:
from sklearn.feature_extraction.text import TfidfVectorizer

def answer_four():
    count=TfidfVectorizer()
    X_train_vectorized=count.fit_transform(X_train)
    features=np.array(count.get_feature_names())
    sorted_tfidf_index = X_train_vectorized.max(0).toarray()[0].argsort()
    smallest=features[sorted_tfidf_index[0:20]]
    largest=features[sorted_tfidf_index[-20:]]
    tfidf=X_train_vectorized.max(0).toarray()[0]
    tfidf.sort()
    smallest_tfidf=tfidf[0:20]
    largest_tfidf=tfidf[-20:]
    s=pd.Series(smallest_tfidf,index=smallest)
    s=s.iloc[np.lexsort([s.index,s.values])]
    l=pd.Series(largest_tfidf,index=largest)
    l=l.iloc[np.lexsort([l.index,l.values])]

    return (s,l)

### Question 5

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **3**.

Then fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1` and compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [132]:
def answer_five():
    count=TfidfVectorizer(min_df=3)
    X_train_vectorized=count.fit_transform(X_train)

    model=MultinomialNB(alpha=0.1)
    model.fit(X_train_vectorized,y_train)
    predictions=model.predict(count.transform(X_test))
    return roc_auc_score(y_test,predictions)
    


### Question 6

What is the average length of documents (number of characters) for not spam and spam documents?

*This function should return a tuple (average length not spam, average length spam).*

In [133]:
def answer_six():
    notspam=spam_data[spam_data['target']==0]['text']
    spam=spam_data[spam_data['target']==1]['text']
    len_spam=np.mean([i for i in map(len,spam)])
    len_not_spam=np.mean([i for i in map(len,notspam)])
    
    return (len_not_spam,len_spam)

<br>
<br>
The following function has been provided to help you combine new features into the training data:

In [134]:
def add_feature(X, feature_to_add):
    """
    Returns sparse feature matrix with added feature.
    feature_to_add can also be a list of features.
    """
    from scipy.sparse import csr_matrix, hstack
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')

### Question 7

Fit and transform the training data X_train using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5**.

Using this document-term matrix and an additional feature, **the length of document (number of characters)**, fit a Support Vector Classification model with regularization `C=10000`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [135]:
from sklearn.svm import SVC

def answer_seven():
    count=TfidfVectorizer(min_df=5)
    X_train_vectorized=count.fit_transform(X_train)
    X_test_vectorized=count.transform(X_test)
    train_add=[i for i in map(len,X_train)]
    test_add=[i for i in map(len,X_test)]
    X_train_vectorized=add_feature(X_train_vectorized,train_add)
    X_test_vectorized=add_feature(X_test_vectorized,test_add)

    model=SVC(C=10000)
    model.fit(X_train_vectorized,y_train)
    predictions=model.predict(X_test_vectorized)
    auc=roc_auc_score(y_test,predictions)
    
    return auc

### Question 8

What is the average number of digits per document for not spam and spam documents?

*This function should return a tuple (average # digits not spam, average # digits spam).*

In [136]:
def answer_eight():
    notspam=spam_data[spam_data['target']==0]['text']
    spam=spam_data[spam_data['target']==1]['text']
    char_spam=[i for i in map(list,spam)]
    char_not_spam=[i for i in map(list,notspam)]
    spam_mean_digits=np.mean([j for j in map(lambda x: sum([i for i in map(str.isdigit,x)]),char_spam)])
    notspam_mean_digits=np.mean([j for j in map(lambda x: sum([i for i in map(str.isdigit,x)]),char_not_spam)])
    
    return (notspam_mean_digits,spam_mean_digits)

### Question 9

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **word n-grams from n=1 to n=3** (unigrams, bigrams, and trigrams).

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* **number of digits per document**

fit a Logistic Regression model with regularization `C=100`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [137]:
from sklearn.linear_model import LogisticRegression

def answer_nine():
    
    count=TfidfVectorizer(min_df=5,ngram_range=(1,3))
    X_train_vectorized=count.fit_transform(X_train)
    X_test_vectorized=count.transform(X_test)

    train_len=[i for i in map(len,X_train)]
    test_len=[i for i in map(len,X_test)]

    char_train=[i for i in map(list,X_train)]
    char_test=[i for i in map(list,X_test)]
    train_digits=[j for j in map(lambda x: sum([i for i in map(str.isdigit,x)]),char_train)]
    test_digits=[j for j in map(lambda x: sum([i for i in map(str.isdigit,x)]),char_test)]

    X_train_vectorized=add_feature(X_train_vectorized,[train_len,train_digits])
    X_test_vectorized=add_feature(X_test_vectorized,[test_len,test_digits])

    model=LogisticRegression(C=100)
    model.fit(X_train_vectorized,y_train)
    predictions=model.predict(X_test_vectorized)
    auc=roc_auc_score(y_test,predictions)


    return auc

### Question 10

What is the average number of non-word characters (anything other than a letter, digit or underscore) per document for not spam and spam documents?

*Hint: Use `\w` and `\W` character classes*

*This function should return a tuple (average # non-word characters not spam, average # non-word characters spam).*

In [122]:
def answer_ten():
    
    import re
    notspam=spam_data[spam_data['target']==0]['text']
    spam=spam_data[spam_data['target']==1]['text']
    char_spam=[i for i in map(list,spam)]
    char_not_spam=[i for i in map(list,notspam)]
    spam_nonwords=np.mean([j for j in map(lambda x: sum([i for i in map(lambda y: bool(re.search(r'\W',y)),x)]),char_spam)])
    nonspam_nonwords=np.mean([j for j in map(lambda x: sum([i for i in map(lambda y:bool(re.search(r'\W',y)),x)]),char_not_spam)])
    return (nonspam_nonwords,spam_nonwords)

### Question 11

Fit and transform the training data X_train using a Count Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **character n-grams from n=2 to n=5.**

To tell Count Vectorizer to use character n-grams pass in `analyzer='char_wb'` which creates character n-grams only from text inside word boundaries. This should make the model more robust to spelling mistakes.

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* number of digits per document
* **number of non-word characters (anything other than a letter, digit or underscore.)**

fit a Logistic Regression model with regularization C=100. Then compute the area under the curve (AUC) score using the transformed test data.

Also **find the 10 smallest and 10 largest coefficients from the model** and return them along with the AUC score in a tuple.

The list of 10 smallest coefficients should be sorted smallest first, the list of 10 largest coefficients should be sorted largest first.

The three features that were added to the document term matrix should have the following names should they appear in the list of coefficients:
['length_of_doc', 'digit_count', 'non_word_char_count']

*This function should return a tuple `(AUC score as a float, smallest coefs list, largest coefs list)`.*

In [123]:
def answer_eleven():
    temp = spam_data.copy()
    temp['length_of_doc'] = temp['text'].str.len()
    temp['digit_count'] = spam_data['text'].apply(lambda row: len(re.findall(r'\d', row)))
    temp['non_word_char_count'] = temp['text'].apply(lambda row: len(re.findall(r'\W', row)))
    X_train, X_test, y_train, y_test = train_test_split(temp.drop('target', axis=1), temp['target'], random_state=0)
    
    vect = CountVectorizer(min_df=5, ngram_range=(2, 5), analyzer='char_wb').fit(X_train['text'])
    X_train_vectorized = vect.transform(X_train['text'])
    X_test_vectorized = vect.transform(X_test['text'])
    X_train_vectorized = add_feature(X_train_vectorized, X_train['length_of_doc'])
    X_train_vectorized = add_feature(X_train_vectorized, X_train['digit_count'])
    X_train_vectorized = add_feature(X_train_vectorized, X_train['non_word_char_count'])
    X_test_vectorized = add_feature(X_test_vectorized, X_test['length_of_doc'])
    X_test_vectorized = add_feature(X_test_vectorized, X_test['digit_count'])
    X_test_vectorized = add_feature(X_test_vectorized, X_test['non_word_char_count'])
    clf = LogisticRegression(C=100).fit(X_train_vectorized, y_train)
    y_score = clf.predict(X_test_vectorized)
    score = roc_auc_score(y_test, y_score)
    
    feature_names = np.append(np.array(vect.get_feature_names()), ['length_of_doc', 'digit_count', 'non_word_char_count'])
    sorted_coef_index = clf.coef_[0].argsort()
    largest_coefs = feature_names[sorted_coef_index[:-11:-1]]
    smallest_coefs = feature_names[sorted_coef_index[:10]]
    
    return score, list(smallest_coefs), list(largest_coefs)

In [139]:
temp = spam_data.copy()
temp['length_of_doc'] = temp['text'].str.len()
temp['digit_count'] = spam_data['text'].apply(lambda row: len(re.findall(r'\d', row)))
temp['non_word_char_count'] = temp['text'].apply(lambda row: len(re.findall(r'\W', row)))
X_train, X_test, y_train, y_test = train_test_split(temp.drop('target', axis=1), temp['target'], random_state=0)
    
vect = CountVectorizer(min_df=5, ngram_range=(2, 5), analyzer='char_wb').fit(X_train['text'])
X_train_vectorized = vect.transform(X_train['text'])
X_test_vectorized = vect.transform(X_test['text'])

In [142]:
[i for i in map(len,vect.get_feature_names())]

[2,
 3,
 3,
 4,
 4,
 5,
 2,
 2,
 3,
 2,
 3,
 3,
 4,
 5,
 3,
 4,
 5,
 2,
 3,
 3,
 4,
 4,
 5,
 3,
 4,
 3,
 2,
 3,
 4,
 3,
 3,
 4,
 3,
 2,
 3,
 3,
 3,
 4,
 3,
 4,
 5,
 3,
 4,
 2,
 3,
 2,
 3,
 2,
 3,
 2,
 3,
 3,
 4,
 4,
 5,
 5,
 2,
 2,
 3,
 4,
 3,
 4,
 5,
 4,
 3,
 4,
 4,
 3,
 4,
 5,
 5,
 4,
 5,
 4,
 5,
 5,
 3,
 4,
 5,
 5,
 5,
 2,
 3,
 3,
 3,
 4,
 4,
 3,
 3,
 4,
 4,
 5,
 5,
 4,
 5,
 4,
 5,
 3,
 4,
 4,
 5,
 3,
 4,
 4,
 5,
 3,
 4,
 5,
 3,
 3,
 4,
 5,
 3,
 4,
 4,
 5,
 3,
 4,
 4,
 5,
 3,
 4,
 5,
 3,
 4,
 5,
 2,
 3,
 3,
 3,
 4,
 3,
 4,
 4,
 5,
 5,
 4,
 5,
 3,
 3,
 3,
 4,
 5,
 3,
 3,
 3,
 4,
 5,
 3,
 3,
 4,
 5,
 3,
 4,
 5,
 4,
 5,
 3,
 4,
 5,
 2,
 3,
 3,
 3,
 4,
 3,
 4,
 4,
 5,
 3,
 3,
 4,
 5,
 3,
 4,
 5,
 3,
 4,
 3,
 4,
 2,
 3,
 3,
 4,
 3,
 4,
 3,
 3,
 3,
 3,
 4,
 3,
 2,
 3,
 3,
 3,
 4,
 5,
 3,
 4,
 5,
 4,
 2,
 3,
 3,
 4,
 3,
 3,
 3,
 4,
 4,
 2,
 3,
 3,
 4,
 3,
 4,
 5,
 3,
 2,
 3,
 3,
 3,
 4,
 5,
 5,
 3,
 3,
 4,
 5,
 3,
 4,
 3,
 4,
 3,
 4,
 5,
 3,
 4,
 5,
 3,
 4,
 5,
 5,
 4,
 3,
 4,
 3,
 3,
 4,
